<a href="https://colab.research.google.com/github/markusloecher/DataScience2021/blob/main/TWSM/Class5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL"

#from TWSM import *
from tensorflow import keras
from tensorflow.keras import layers
import spacy
## Import packages
import pickle
import pandas as pd
from gensim.models import Word2Vec

# run this from a normal command line
#!python -m spacy download en_core_web_md #160MB

#can I download this to a local file instead and load it fom drive?


Mounted at /content/drive


In [50]:
import numpy as np

In [ ]:
!pip install whatlies

     |████████████████████████████████| 85 kB 1.6 MB/s 
     |████████████████████████████████| 24.2 MB 2.3 MB/s 
     |████████████████████████████████| 1.2 MB 40.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


https://stackoverflow.com/questions/56927602/unable-to-load-the-spacy-model-en-core-web-lg-on-google-colab

Now, *** restart the colab runtime *** !!

## word2vec in spacy

In [ ]:
import spacy
# Load the spacy model that you have installed
nlp = spacy.load('en_core_web_md')

# process a sentence using the model
doc = nlp("The sun is shining brightly today but the moon is not")

# It's that simple - all of the vectors and words are assigned after this point
# Get the vector for 'text':
doc[3].vector

# Get the mean vector for the entire sentence (useful for sentence classification etc.)
doc.vector

OSError: ignored

In [ ]:
#Opposites are not necessarily different
doc = nlp("I loved Narnia but hated Armageddon")

print(doc[1])
print(doc[4])
print(doc[1].similarity(doc[4]))
print(doc[4].similarity(doc[1]))

doc = nlp("The king and the queen are enjoying a sumptious breakfast today")

print(doc[1])
print(doc[4])
print(doc[1].similarity(doc[4]))
print(doc[4].similarity(doc[1]))

loved
hated
0.66889775
0.66889775
king
queen
0.72526103
0.72526103


WhatLies module

In [ ]:
king = nlp.vocab["king"].vector
man = nlp.vocab["man"].vector
queen = nlp.vocab["queen"].vector
woman = nlp.vocab["woman"].vector

#or:
def w2v(w="king"):
  return nlp.vocab[w].vector

king = w2v("king")

In [ ]:
from whatlies import EmbeddingSet
from whatlies.language import SpacyLanguage

lang = SpacyLanguage('en_core_web_md')
words = ['cat', 'dog', 'fish', 'kitten', 'man', 'woman', 'king', 'queen', 'doctor', 'nurse']

emb = lang[words]
emb.plot_interactive(x_axis='man', y_axis='woman')

alt.LayerChart(...)

**Vector Algebra**

seems much more difficult than in gensim

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

queries = [w for w in nlp.vocab if w.is_lower and w.prob >= -15]

def cos_sim(a, b):
    return cosine_similarity(a.reshape(1, -1), b.reshape(1, -1))

def most_similar_vec(vec, count=10):
    by_similarity = sorted(queries, key=lambda w: cos_sim(w.vector, vec), reverse=True)
    return [w.orth_ for w in by_similarity[:count]]

vec = nlp('woman').vector + nlp('king').vector - nlp("man").vector
most_similar_vec(vec)

['king',
 'queen',
 'prince',
 'princes',
 'kings',
 'princess',
 'princesses',
 'mermaid',
 'royal',
 'royals']

In [ ]:
queries

## word2vec in gensim

In [ ]:
import gensim.downloader as api
#https://github.com/RaRe-Technologies/gensim-data
wv = api.load('glove-wiki-gigaword-100')#128MB
#wv = api.load('glove-wiki-gigaword-50')#65MB


[==================================================] 100.0% 128.1/128.1MB downloaded


In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.67
'car'	'bicycle'	0.69
'car'	'airplane'	0.65
'car'	'cereal'	0.12
'car'	'communism'	0.04


## Data

**IMD Movie Reviews**

In [3]:
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(
    num_words=10000)

17473536/17464789 [==============================] - 0s 0us/step


In [1]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = "/content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL"

#from TWSM import *
from tensorflow import keras
from tensorflow.keras import layers
import spacy
## Import packages
import pickle
import pandas as pd
from gensim.models import Word2Vec
import spacy

# run this from a normal command line
#!python -m spacy download en_core_web_md #160MB

Mounted at /content/drive


#### Tasks

1. Train a word2vec model of dimension $100$ on the IMD data. (considering words appearing in more than 50 documents) Save the model if you like.

2. Compute the embedding for each review (average word2vec)

3. Fit a keras classifier to the embedded reviews. (2 hidden layers of size 40 each) Report/Monitor the accuracy on the test data.

4. Load the bing sentiment dictionary. Compute two separate embeddings for the negative and positive sentiments.

5. Compute the similarity between these two vectors and a few selected reviews. Does it agree with their label?


# **Examine Data**

In [17]:
print(train_data.shape)

(25000,)


In [18]:
print(train_labels.shape)

(25000,)


# **Reviews back to text**

In [22]:
word_index = imdb.get_word_index()
reverse_word_index = dict(
    [(value, key) for (key, value) in word_index.items()])
decoded_reviews = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in train_data[0]])

decoded_reviews

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [23]:
N=len(train_data)
decoded_reviews = ["" for x in range(N)]

for j in range(N):
  decoded_reviews[j] = " ".join(
    [reverse_word_index.get(i - 3, "?") for i in train_data[j]])

In [30]:
decoded_reviews[1]

"? big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ? the hair is big lots of boobs ? men wear those cut ? shirts that show off their ? sickening that men actually wore them and the music is just ? trash that plays over and over again in almost every scene there is trashy music boobs and ? taking away bodies and the gym still doesn't close for ? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

# **Model**

In [24]:
corpus=[doc.split() for doc in decoded_reviews]

In [25]:
model = Word2Vec(corpus, size=100, min_count=50)

In [31]:
model.save("word2vec.model")
print(model)

Word2Vec(vocab=7261, size=100, alpha=0.025)


# **Average Word2vec**

In [35]:
# Document representation for the text
corpus_w2v=[[model.wv[word] for word in doc if word in model.wv.vocab.keys()] for doc in corpus]
positive=[i for i in range(len(corpus)) if len(corpus_w2v[i])>0]

corpus_w2v2=[corpus_w2v[i] for i in positive]

# Document average representation
corpus_w2v_avg_clean=[sum(words)/len(words) for words in corpus_w2v2]

# This corpus can be used later in clustering and classification tasks
print(corpus_w2v_avg_clean[10])

[-0.13179548 -0.13428992 -0.18508519  0.05798002  0.11014589 -0.08045991
 -0.28172222  0.09762453  0.2570829  -0.03372364  0.2264569   0.17756997
  0.39125568 -0.16930884 -0.2592233   0.5050119   0.25508517 -0.22835918
  0.03023599 -0.27985355  0.00683441  0.03405618  0.12408944 -0.28738195
 -0.18141137 -0.12173939 -0.099632    0.02689914 -0.06626983 -0.2208648
 -0.3282246   0.22681588 -0.18801802 -0.2329783  -0.20070703  0.52878934
 -0.16635163  0.20430443 -0.1351676   0.6688067  -0.5507718   0.41877732
  0.04052142  0.17801255  0.2701071  -0.33845407  0.05278147 -0.0515018
 -0.33697695  0.56693256 -0.21323766  0.03803716  0.15625826  0.03573452
 -0.44171822  0.01271552  0.23872529  0.11314425 -0.07062407 -0.08412804
 -0.00600381  0.37189963 -0.11365755  0.3199189   0.04081683 -0.08059935
 -0.13741228 -0.10910499  0.05957192  0.15013348 -0.27011198 -0.04556013
  0.08208993  0.03683763  0.05591073 -0.01929143 -0.3782493   0.42942774
 -0.14842434  0.2834748  -0.12511815 -0.32975313 -0.1

In [36]:
len(corpus_w2v)

25000

In [37]:
len(corpus_w2v_avg_clean)

25000

In [39]:
model.wv.similar_by_vector(corpus_w2v_avg_clean[0])

[('it', 0.7002661824226379),
 ('really', 0.6868996024131775),
 ('just', 0.6563368439674377),
 ('actually', 0.6291176080703735),
 ('this', 0.6163733005523682),
 ('bad', 0.600469708442688),
 ('disappointed', 0.598484456539154),
 ('movie', 0.592471182346344),
 ('good', 0.5775361061096191),
 ('funny', 0.5705164670944214)]

In [43]:
# Corpus as data frame that can be used in downstream tasks such as classification
corpus_w2v_avg_df=pd.DataFrame(corpus_w2v_avg_clean)
ecorpus_w2v_avg_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.002547,-0.053586,0.110606,0.271018,0.018488,-0.312736,-0.358602,0.001325,0.559362,-0.102771,...,0.215092,-0.482986,-0.038197,0.389945,0.509781,0.136070,0.007076,-0.032305,1.053163,-0.154776
1,-0.002277,-0.166709,-0.021454,0.282407,0.232869,-0.284560,-0.392670,0.136791,0.196933,0.140207,...,0.220437,-0.061238,-0.302896,0.257258,0.278716,0.124231,-0.345381,0.105101,0.775254,-0.016696
2,0.086036,0.164436,0.080638,0.153089,-0.100223,0.060261,-0.331279,0.094121,0.291988,0.150128,...,0.049495,-0.375090,-0.050422,0.489581,0.719987,0.246373,-0.062145,-0.046875,1.023154,-0.105591
3,-0.204331,0.111560,-0.181981,-0.206662,0.001540,-0.224015,0.014030,0.157554,0.185863,-0.049888,...,0.193298,-0.182343,-0.167690,0.136011,0.197434,0.073042,-0.128013,-0.019613,0.726028,0.011240
4,-0.160111,0.214807,-0.030208,0.059089,-0.440250,-0.041743,-0.216256,-0.013227,0.324131,0.271408,...,0.166917,-0.423361,0.022852,0.541207,0.496034,0.330351,-0.211074,0.046276,1.250896,-0.075437


# **Keras Classifier**

In [44]:
clf = keras.Sequential()
clf.add(layers.Dense(40))
clf.add(layers.Dense(40))

In [57]:
clf.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [58]:
clf.fit(np.array(corpus_w2v_avg_clean),
                    train_labels,
                    epochs=20,
                    validation_split=0.3)

Epoch 1/20
547/547 [==============================] - 2s 3ms/step - loss: 0.9951 - accuracy: 0.0014 - val_loss: 0.5076 - val_accuracy: 0.0000e+00
Epoch 2/20
547/547 [==============================] - 1s 2ms/step - loss: 0.4790 - accuracy: 0.0000e+00 - val_loss: 0.4707 - val_accuracy: 0.0000e+00
Epoch 3/20
547/547 [==============================] - 1s 2ms/step - loss: 0.4596 - accuracy: 0.0013 - val_loss: 0.4508 - val_accuracy: 0.0052
Epoch 4/20
547/547 [==============================] - 1s 2ms/step - loss: 0.5043 - accuracy: 0.0013 - val_loss: 0.4453 - val_accuracy: 4.0000e-04
Epoch 5/20
547/547 [==============================] - 1s 2ms/step - loss: 0.9169 - accuracy: 0.0187 - val_loss: 7.7238 - val_accuracy: 0.0000e+00
Epoch 6/20
547/547 [==============================] - 1s 2ms/step - loss: 7.5766 - accuracy: 0.0000e+00 - val_loss: 7.7223 - val_accuracy: 0.0000e+00
Epoch 7/20
547/547 [==============================] - 1s 2ms/step - loss: 7.5718 - accuracy: 0.0000e+00 - val_loss: 7.71

In [54]:
clf.evaluate

<bound method Model.evaluate of <keras.engine.sequential.Sequential object at 0x7fc7d18e5f90>>

In [ ]:
# train test???

# **Bing**

In [62]:
bing_df = pd.read_csv("/content/drive/Othercomputers/Mi portátil/Master/GitHub/DataScience2021/TWSM/bing.csv", index_col=0)
bing_df.head()

,word,sentiment
1,2-faces,negative
2,abnormal,negative
3,abolish,negative
4,abominable,negative
5,abominably,negative


In [93]:
bing_pos = bing_df[bing_df["sentiment"] == "positive"]
bing_pos = bing_pos[["word"]]

In [94]:
bing_neg = bing_df[bing_df["sentiment"] == "negative"]
bing_neg = bing_neg[["word"]]

In [97]:
decoded_reviews[1]

"? big hair big boobs bad music and a giant safety pin these are the words to best describe this terrible movie i love cheesy horror movies and i've seen hundreds but this had got to be on of the worst ever made the plot is paper thin and ridiculous the acting is an abomination the script is completely laughable the best is the end showdown with the cop and how he worked out who the killer is it's just so damn terribly written the clothes are sickening and funny in equal ? the hair is big lots of boobs ? men wear those cut ? shirts that show off their ? sickening that men actually wore them and the music is just ? trash that plays over and over again in almost every scene there is trashy music boobs and ? taking away bodies and the gym still doesn't close for ? all joking aside this is a truly bad film whose only charm is to look back on the disaster that was the 80's and have a good old laugh at how bad everything was back then"

In [99]:
doc = clf(decoded_reviews[1])

TypeError: ignored